In [1]:
import pandas as pd
import time

In [2]:
file_start_time = time.time()

In [3]:
header = ['_id', 'type', 'mmsi', 'status', 'turn', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'time', 'imo', 'callsign', 'shipname', 'shiptype', 'to_bow',
          'to_stern', 'to_port', 'to_starboard', 'month', 'day', 'hour', 'minute', 'draught', 'destination', 'destination_country',	'shiptype_vessel',	
          'vessel_flag', 'vessel_flag_continent',' vessel_flag_country_code\r']
        # tbc header name from country_destination

In [4]:
start_time = time.time()

chunk = pd.read_csv('cleaned_type5.csv',chunksize=100000,lineterminator='\n')
df = pd.concat(chunk)

# start_time = time.time()
end_time = time.time()
read_time =   end_time - start_time
print("Read time:",read_time)


Read time: 2.7133119106292725


In [5]:
df.head()

,_id,type,mmsi,time,imo,callsign,shipname,shiptype,to_bow,to_stern,to_port,to_starboard,draught,destination,destination_country,shiptype_vessel,vessel_flag,vessel_flag_continent,vessel_flag_country_code\r
0,6533fb51281b73e6e0bd037b,5,205559000,2023-09-09 18:07:29+08,9416733,ONGB,FRATERNITY,89,230,44,17,31,13.7,SGSIN,SINGAPORE,Tanker,Belgium,EUROPE,BE\r
1,6533fb4b281b73e6e0bb4805,5,205083000,2023-09-09 15:35:18+08,9792527,ORUE,BOKA TIAMAT,79,40,58,10,10,5.2,SGSIN PWBGB,SINGAPORE,Cargo,Belgium,EUROPE,BE\r
2,6533fb3e281b73e6e0b7b5e4,5,205559000,2023-09-09 09:31:29+08,9416733,ONGB,FRATERNITY,89,230,44,17,31,16.9,SGSIN,SINGAPORE,Tanker,Belgium,EUROPE,BE\r
3,6533fb38281b73e6e0b5e184,5,205559000,2023-09-09 06:37:27+08,9416733,ONGB,FRATERNITY,89,230,44,17,31,16.9,SGSIN,SINGAPORE,Tanker,Belgium,EUROPE,BE\r
4,6533fb3c281b73e6e0b6f14c,5,205559000,2023-09-09 08:19:25+08,9416733,ONGB,FRATERNITY,89,230,44,17,31,16.9,SGSIN,SINGAPORE,Tanker,Belgium,EUROPE,BE\r


In [6]:
start_time = time.time()
df.isna().sum()
end_time = time.time()
execution_time =   end_time - start_time
print("Execution time:",execution_time)

Execution time: 0.3024575710296631


In [7]:
df.shape

(1417249, 19)

# Cleaning steps
1. Drop message id col
2. Split by msg type (123, 5, 1819)
3. Remove MMSI less than 9 digits
4. Extract SG destination
   
For 123 and 1819:
1. convert status to int

For 5:

# NA
1. Callsign drop non alpha-numeric (callsign ~ '^[A-Za-z0-9]+$'  and > 0)
2. Convert shiptype to int
4. Drop month, day, hour minute


In [8]:
clean_start_time = time.time()

In [9]:
df_clean = df.drop('_id', axis = 1)

In [10]:
df_clean.shape

(1417249, 18)

In [11]:
# clean mmsi shorter than 9 digits
df_clean = df_clean[df_clean.mmsi > 99999999]

In [12]:
df_clean.shape

(1417136, 18)

In [13]:
# start_time = time.time()
# df_type123 = df_clean[df_clean.type <= 3][['mmsi', 'status', 'turn', 'speed', 'accuracy', 'lon', 'lat', 'course', 'heading', 'time']]
# print(df_type123.shape)
# end_time = time.time()
# execution_time =   end_time - start_time
# print("Execution time:",execution_time)

In [14]:
start_time = time.time()
df_type5 = df_clean[df_clean.type == 5][['mmsi', 'time', 'draught', 'destination_country']] # cleaned destination 
print(df_type5.shape)
end_time = time.time()
execution_time =   end_time - start_time
print("Execution time:",execution_time)

(1417136, 4)
Execution time: 0.3599581718444824


In [15]:
# start_time = time.time()
# df_type1819 = df_clean[df_clean.type >= 18][['mmsi', 'speed', 'accuracy', 'lon', 'lat', 'course', 'time']]
# print(df_type1819.shape)
# end_time = time.time()
# execution_time =   end_time - start_time
# print("Execution time:",execution_time)

# Clean df_type 123

In [16]:
# # change status to type int
# df_type123['status'] = df_type123['status'].astype(int)

# Clean df_type 5

# Clean df_type 1819

In [17]:
clean_end_time = time.time()
clean_time =   clean_end_time - clean_start_time
print("Cleaning time:",clean_time)

Cleaning time: 0.7735941410064697


# Destination cleaning (NA)

In [18]:
# # Singapore location cleaning
# keywords = ['sg', 'singapore', 'sin', 'sgp', 'PEBGA', 'PEBGB', 'PSBG', 'PWBGA', 'PWBGB', 'PJSB', 'PGBG', 'ACBTH', 'ACGP', 'AEBA', 'AEBB', 'AEBC', 'AEPB', 'AEPBB', 'AEPBC', 'AESPA', 'ASCA', 'ASCB', 'AMOW', 'AEW', 'AEPA', 'AEHA', 'AEHC', 'AEHB']

# # Function to clean destination data using input
# def clean_destination(destination):
#     destination = str(destination)
#     cleaned_destination = [word.upper() for word in keywords if word.lower() in destination.lower()]
#     if cleaned_destination:
#         return 'SG'
        
# # Apply the function to the 'destination' column and create a new column 'destination_clean'
# df_clean['destination_clean'] = df_clean['destination'].apply(clean_destination)

# # drop destination column
# # df_clean = df_clean.drop('destination', axis = 1)

In [19]:
# df_clean.destination_clean.value_counts()

In [20]:
# countries_keywords = {
#     'SG': ['sg', 'singapore', 'sin', 'sgp'],  # Singapore
#     'US': ['us', 'usa', 'america'],  # United States
#     'CN': ['cn', 'china', 'prc'],  # China
#     'JP': ['jp', 'japan'],  # Japan
#     'KR': ['kr', 'south korea'],  # South Korea
#     'DE': ['de', 'germany'],  # Germany
#     'NL': ['nl', 'netherlands'],  # Netherlands
#     'AE': ['ae', 'united arab emirates', 'uae'],  # United Arab Emirates
#     'HK': ['hk', 'hong kong'],  # Hong Kong
#     'MY': ['my', 'malaysia'],  # Malaysia
#     'BR': ['br', 'brazil'],  # Brazil
#     'AU': ['au', 'australia'],  # Australia
#     'ID': ['id', 'indonesia'],  # Indonesia
#     'TH': ['th', 'thailand'],  # Thailand
#     'VN': ['vn', 'vietnam'],  # Vietnam
#     'PH': ['ph', 'philippines'],  # Philippines
#     'MM': ['mm', 'myanmar'],  # Myanmar
#     'KH': ['kh', 'cambodia'],  # Cambodia
#     'LA': ['la', 'lao', 'laos'],  # Laos
#     'BN': ['bn', 'brunei'],  # Brunei
#     'TL': ['tl', 'timor-leste'],  # Timor-Leste
#     'IN': ['in', 'india'],  # India
#     'PK': ['pk', 'pakistan'],  # Pakistan
#     'BD': ['bd', 'bangladesh'],  # Bangladesh
#     'LK': ['lk', 'sri lanka'],  # Sri Lanka
#     'NP': ['np', 'nepal'],  # Nepal
#     'MM': ['mm', 'myanmar'],  # Myanmar
#     # Add more countries and their keywords as needed
# }

In [21]:
# start_time = time.time()

# for code, keywords in countries_keywords.items():
#     def clean_destination(destination):
#         destination = str(destination)
#         cleaned_destination = [word.upper() for word in keywords if word.lower() in destination.lower()]
#         if cleaned_destination:
#             return code

#     df_clean['destination_clean'] = df_clean['destination'].apply(clean_destination)

# end_time = time.time()
# execution_time =   end_time - start_time
# print("Execution time:",execution_time)
# df_clean.destination_clean.value_counts()

In [22]:
# def clean_destination(row):
#     destination = str(row['destination'].lower())
#     for code, keywords in countries_keywords.items():
#         if any(keyword in destination for keyword in keywords):
#             return code.upper()
#     return None

In [23]:
# # Apply the clean_destination function to create 'destination_clean' column
# df_clean['destination_clean'] = df_clean.apply(clean_destination, axis=1)

In [24]:
# df_clean.destination_clean.value_counts()

# All cleaning steps completed, save to csv

In [25]:
# df_clean.to_csv('df_clean.csv')

In [26]:
# df_type123.to_csv('df_type123.csv')

In [27]:
df_type5.to_csv('df_type5.csv')

In [30]:
df_type5.head()

,mmsi,time,draught,destination_country
0,205559000,2023-09-09 18:07:29+08,13.7,SINGAPORE
1,205083000,2023-09-09 15:35:18+08,5.2,SINGAPORE
2,205559000,2023-09-09 09:31:29+08,16.9,SINGAPORE
3,205559000,2023-09-09 06:37:27+08,16.9,SINGAPORE
4,205559000,2023-09-09 08:19:25+08,16.9,SINGAPORE


In [28]:
# df_type1819.to_csv('df_type1819.csv')

In [29]:
file_end_time = time.time()
file_execution_time = file_end_time - file_start_time
print("Read time:",read_time)
print("Cleaning time:",clean_time)
print("File run time:",file_execution_time)

Read time: 2.7133119106292725
Cleaning time: 0.7735941410064697
File run time: 6.264193534851074
